In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, LSTM, TimeDistributed, Reshape
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import joblib


In [ ]:
train_labels=pd.read_csv("/kaggle/input/mlware24/train-labels_mlware.csv")

In [ ]:
train_labels.shape

In [ ]:
train_labels.info()

In [ ]:
train_labels.head()

In [ ]:
train_labels.drop("Unnamed: 0",axis=1)

In [ ]:
def load_image(img_path):
    return Image.open(img_path)

In [ ]:
data = {}
for index, row in train_labels.iterrows():
    # Get the image ID and label
    image_id = row['image']
    label = row['text']

    # Load the image
    images = load_image(f'/kaggle/input/mlware24/train_images_mlware/train_images/{image_id}')

    # Store the image and label in the dictionary
    data[image_id] = {'image': images, 'label': label}

# Print the first 5 items in the dictionary to verify the data
for key in list(data.keys())[:5]:
    print(key, data[key]['label'])

In [ ]:


# Specify the directory where the images are
image_dir = '/kaggle/input/mlware24/train_images_mlware/train_images/'

# Get the list of image file names
image_files = os.listdir(image_dir)

# Sort the image files
image_files.sort()

# Display the first 5 images
for i in range(5):
    # Open the image file
    image = Image.open(os.path.join(image_dir, image_files[i]))
    
    # Display the image
    plt.imshow(image)
    plt.title(image_files[i])
    plt.show()


In [ ]:
import numpy as np
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

In [ ]:
X = np.array([img_to_array(data[key]['image'].convert('L').resize((50, 50))) for key in data.keys()])
X = X.astype('float32') / 255

# Encode labels using one-hot encoding
lb = LabelBinarizer().fit(list(data[key]['label'] for key in data.keys()))
y = lb.transform(list(data[key]['label'] for key in data.keys()))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define the model
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(50, 50, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(len(lb.classes_), activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
# Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

In [ ]:
# Make a prediction for the first image in the test set
prediction = model.predict(np.array([X_test[0]]))

# The prediction is a one-hot encoded array, so we'll use the inverse_transform method to get the original label
predicted_label = lb.inverse_transform(prediction)

print('Predicted label:', predicted_label)
print('Actual label:', lb.inverse_transform(np.array([y_test[0]])))


**so this code did'nt worked properly will do some more changes in the images as it is a graysclae image need to do some augumentation**

In [ ]:
# Convert images to grayscale and resize them
X = np.array([img_to_array(data[key]['image'].convert('L').resize((128, 128))) for key in data.keys()])
X = X.astype('float32') / 255

In [ ]:
X[:2]

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer


# Flatten the array of labels
labels = [data[key]['label'] for key in data.keys()]

# Initialize the tokenizer
tokenizer = Tokenizer(char_level=True,lower=False)

# Fit the tokenizer on the labels
tokenizer.fit_on_texts(labels)

# Transform the labels to one-hot encoded vectors
y = np.array([tokenizer.texts_to_matrix(label) for label in labels])


In [ ]:
y[:2]

In [ ]:
from keras.layers import LSTM, TimeDistributed, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.models import Sequential

# Define the model

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)))  # Changed from (128, 128, 3) to (128, 128, 1)
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(1020, activation='relu'))
model.add(Reshape((6, -1)))  # Reshape the output into sequences
model.add(LSTM(128, return_sequences=True))
model.add(TimeDistributed(Dense(len(tokenizer.word_index) + 1, activation='softmax')))


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32)

In [ ]:
test_dir = '/kaggle/input/mlware24/test_images_mlware/test_images/'

test_files = os.listdir(test_dir)



In [ ]:
predictions = []

for file in test_files:
    image = Image.open(os.path.join(test_dir, file)).convert('L').resize((128, 128))
    
    image = img_to_array(image) / 255.0
    
    prediction = model.predict(np.expand_dims(image, axis=0))
    
    predicted_label = ''.join(tokenizer.sequences_to_texts(prediction.argmax(axis=-1))).replace(' ', '')

    predictions.append(predicted_label)


In [ ]:
# Specify the directory where the images are
image_dir = '/kaggle/input/mlware24/test_images_mlware/test_images/'

# Get the list of image file names
image_files = os.listdir(image_dir)

# Sort the image files
image_files.sort()

# Display the first 5 images
for i in range(5):
    # Open the image file
    image = Image.open(os.path.join(image_dir, image_files[i]))
    
    # Display the image
    plt.imshow(image)
    plt.title(image_files[i])
    plt.show()

In [ ]:
df = pd.DataFrame({
    'Image': test_files,
    'text': predictions
})

# Print the DataFrame
print(df)

In [ ]:
image = Image.open('/kaggle/input/mlware24/test_images_mlware/test_images/test-2144.png')

# Display the image
plt.imshow(image)
plt.show()

In [ ]:
df.to_csv('submission_file_mlware.csv', index=False)